In [ ]:
# !pip install transformers

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch import nn, Tensor
from torch.nn import TransformerEncoder, TransformerEncoderLayer


import torchtext
from torchtext.legacy import data
from torchtext.legacy import datasets

from collections import defaultdict, Counter

from transformers import BertTokenizer, BertModel

import math
from typing import Tuple
import numpy as np

import time
import random
import functools

import matplotlib.pyplot as plt

%config InlineBackend.figure_format = 'retina'
plt.style.use('seaborn')

In [ ]:
# To ensure that the code is reproducible, set random seeds
SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


In [ ]:
import pandas as pd
pwngc_path = "../data/test_transformer/pwngc4torchtext.csv"
pwngc_df = pd.read_csv(pwngc_path, delimiter="\t", header=None)
pwngc_df.columns = ["token", "stemm", "pos", "annotation", "synset", "tag" ]
pwngc_df.head(20)

In [ ]:
import numpy as np
len(np.where(pwngc_df["tag"] != 'O')[0])
# 532.821 annotated tokens

In [ ]:
import ast
tag2 = pwngc_df['tag'][2]
print(tag2)
print(type(tag2))
def removeBra(string_list):
    if string_list[0] == "[" and string_list[-1] == "]":
        return string_list[1:-1]
    else:
        return string_list

tag2list = torch.tensor(list(map(float, removeBra(tag2).split(' '))), dtype=torch.float32)
# tag2list = ast.literal_eval(tag2)
print(tag2list)
print(type(tag2list))
# type(eval("tensor({}, device='{}')".format(tag2, "cpu")))

In [ ]:
# split the dataset into training, validation and testing
train_path = "train.csv"
validate_path = "validate.csv"
test_path = "test.csv"


In [ ]:
#   train_examples = read_data("../data/test_transformer/train.csv", self.fields) #'data/eng.train.iob', self.fields)

In [ ]:
TEXT = data.Field(use_vocab=True,
                  lower=True)

LABEL = data.Field(is_target=True,
                   use_vocab=False,
                   unk_token=None,
                   preprocessing=data.Pipeline(lambda x: torch.tensor(list(map(float, removeBra(x).split(' '))), dtype=torch.double)),
                   dtype=data.Pipeline(lambda x: torch.tensor(x, dtype=torch.double)))

train, valid, test = datasets.SequenceTaggingDataset.splits(path='../data/test_transformer/',
                                   train = train_path,
                                   validation = validate_path,
                                   test = test_path,
                                   fields=[("text",TEXT),("lemmatized_text",TEXT), (None, None), (None,None), (None, None), ("label",LABEL)]) #,

type(train)

In [ ]:
print(train)

In [ ]:
for t, lt, l in zip(train.text, train.lemmatized_text, train.label):
    print(type(l), type(lt), type(l))
    print(t, lt, l)
    print(len(t))

In [ ]:
for ex in train:
    print(len(ex.text))

In [ ]:
# Load the pre-trained embeddings that come with the torchtext library.
use_pretrained = True
if use_pretrained:
    print('We are using pre-trained word embeddings.')
    TEXT.build_vocab(train, vectors="glove.840B.300d")
else:
    print('We are training word embeddings from scratch.')
    TEXT.build_vocab(train, max_size=5000)

In [ ]:
# for key, val in zip(TEXT.vocab.stoi, TEXT.vocab.vectors):
    # print(key, val)

print(list(train))
# print(TEXT.vocab.stoi)

In [ ]:
# b= data.example.Example.fromlist([train.text, train.label], fields=[("text",TEXT),("lemmatized_text",TEXT), (None, None), (None,None), (None, None), ("label",LABEL)])
# TEXT.build_vocab(b, vectors="glove.840B.300d")
# print(b.vocab)
# for tt in b:
#     print(tt.vocab)

In [ ]:
BATCH_SIZE = 5

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train, valid, test),
    batch_size=BATCH_SIZE,
    # sort=False, # to skip sorting validation and testing data
    sort_key=lambda x: len(x.text),
    repeat=False)

In [ ]:
# print(train_iterator.data())

train_batch = train_iterator.data()
print(TEXT.vocab.freqs.most_common(10))
print("-"*40)
print(TEXT.vocab.itos[:10])
print(TEXT.vocab.stoi)
print(TEXT.vocab.vectors)

In [ ]:
for bat in train_iterator:
    print(bat)
# for batch in train_iterator:
#     print(type(batch))#, type(batch.text), type(batch.label))
    # print(batch.text[0], type(batch.text[0]))
    # print(batch.text, batch.lemmatized_text, batch.label)



# for batch in train_iterator.data():
#     print(type(batch), type(batch.text), type(batch.label))
#     print(batch.text[0], type(batch.text[0]))
#     print(batch.text, batch.lemmatized_text, batch.label)

In [ ]:
list(train_iterator.data())

In [ ]:
train_batches = list(train_iterator.data())
valid_batches = list(valid_iterator.data())

In [ ]:
class PositionalEncoding(nn.Module):

    def __init__(self, d_model: int, dropout: float = 0.1, max_len: int = 5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        position = torch.arange(max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) * (-math.log(10000.0) / d_model))
        pe = torch.zeros(max_len, 1, d_model)
        pe[:, 0, 0::2] = torch.sin(position * div_term)
        pe[:, 0, 1::2] = torch.cos(position * div_term)
        self.register_buffer('pe', pe)

    def forward(self, x):
        """
        Args:
            x: Tensor, shape [seq_len, batch_size, embedding_dim]
        """
        x = x + self.pe[:x.size(0)]
        return self.dropout(x)

In [ ]:
class TransformerEncoderModel(nn.Module):

    def __init__(self, text_field, label_field, ntoken: int, d_model: int, nhead: int, d_hid: int,
                 nlayers: int, dropout: float = 0.5):
        super().__init__()
        self.model_type = 'Transformer'
        self.pos_encoder = PositionalEncoding(d_model, dropout)
        # Multi-head attention mechanism is included in TransformerEncoderLayer
        encoder_layers = TransformerEncoderLayer(d_model, nhead, d_hid, dropout)
        self.transformer_encoder = TransformerEncoder(encoder_layers, nlayers)
        self.encoder = nn.Embedding(ntoken, d_model)
        self.d_model = d_model
        # self.decoder = nn.Linear(d_model, ntoken)

        self.init_weights()
        # -------------------------------------

        voc_size = len(text_field.vocab)
        print("voc_size: ", voc_size )

        # Embedding layer. If we're using pre-trained embeddings, copy them
        # into our embedding module.
        self.embedding = nn.Embedding(voc_size, 300)
        print("Embedding", self.embedding)
        if text_field.vocab.vectors is not None:
            self.embedding.weight = torch.nn.Parameter(TEXT.vocab.vectors)


    def init_weights(self) -> None:
        initrange = 0.1
        self.encoder.weight.data.uniform_(-initrange, initrange)
        # self.decoder.bias.data.zero_()
        # self.decoder.weight.data.uniform_(-initrange, initrange)

    def forward(self, src):
        """
        Args:
            src: Tensor, shape [seq_len, batch_size]

        Returns:
            output Tensor of shape [seq_len, batch_size, ntoken]
        """
        src = torch.tensor(src, device="cpu")
        src = self.embedding(src)
        src = self.encoder(src) * math.sqrt(self.d_model)
        src = self.pos_encoder(src)
        output = self.transformer_encoder(src) #, src_mask)
        # output = self.decoder(output)
        return output


# def generate_square_subsequent_mask(sz: int) -> Tensor:
#     """Generates an upper-triangular matrix of -inf, with zeros on diag."""
#     return torch.triu(torch.ones(sz, sz) * float('-inf'), diagonal=1)

In [ ]:
# train_batches = train_iterator.data()
# for batch in train_batches:
#     print(batch)
#     # batch.text is of shape: (input_data_length, batch_size)
#     print(batch.text)
#     # print(batch.text.vocab)
#     # print(batch.text.vocab.vectors)

In [ ]:
class Tagger:

    def __init__(self, lower):
        self.TEXT = data.Field(use_vocab=True,
                  lower=True)

        self.LABEL = data.Field(is_target=True,
                           use_vocab=False,
                           unk_token=None,
                           preprocessing=data.Pipeline(
                               lambda x: torch.tensor(list(map(float, removeBra(x).split(' '))),
                                                      dtype=torch.double)),
                           dtype=torch.DoubleTensor)
                                #data.Pipeline(lambda x: torch.tensor(x, dtype=torch.double)))

        self.fields = [("text",self.TEXT),("lemmatized_text",self.TEXT), (None, None), (None,None), (None, None), ("label",LABEL)]


        # self.TEXT = data.Field(init_token='<bos>', eos_token='<eos>', sequential=True, lower=lower)
        # I changed sequential = True to false, because my data is not sequential
        # self.LABEL = data.Field(init_token='<bos>', eos_token='<eos>', sequential=True, unk_token=None)
        # self.LABEL = data.Field(is_target=True, sequential=False, unk_token=None, dtype=list)
        # data.Field(init_token='<bos>', eos_token='<eos>',
        #     sequential=False, use_vocab=False)
        # self.fields = [('text', self.TEXT), ('label', self.LABEL)]
        # self.device = 'cuda'
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


    def tag(self, sentences):
        # This method applies the trained model to a list of sentences.

        # First, create a torchtext Dataset containing the sentences to tag.
        examples = []
        for sen in sentences:
            labels = ['?']*len(sen) # placeholder
            examples.append(data.Example.fromlist([sen, labels], self.fields))
        dataset = data.Dataset(examples, self.fields)

        iterator = data.Iterator(
            dataset,
            device=self.device,
            batch_size= 5, #64,
            repeat=False,
            train=False,
            sort=False)

        # Apply the trained model to all batches.
        out = []
        self.model.eval()
        with torch.no_grad():
            for batch in iterator:
                # Call the model's predict method. This returns a list of NumPy matrix
                # containing the integer-encoded tags for each sentence.
                predicted = self.model.predict(batch.text)

                # # Convert the integer-encoded tags to tag strings.
                # for tokens, pred_sen in zip(sentences, predicted):
                #     out.append([self.LABEL.vocab.itos[pred_id] for _, pred_id in zip(tokens, pred_sen[1:])])
        return predicted #out

    def train(self):
        # Read training and validation data according to the predefined split.
        # train_examples = read_data("../data/test_transformer/train.csv", self.fields) #'data/eng.train.iob', self.fields)
        # valid_examples = read_data("../data/test_transformer/validate.csv", self.fields) #'data/eng.valid.iob', self.fields)

        train_examples, valid_examples, test_examples = datasets.SequenceTaggingDataset.splits(path='../data/test_transformer/',
                                           train = train_path,
                                           validation = validate_path,
                                           test = test_path,
                                           fields=[("text",TEXT),("lemmatized_text",TEXT), (None, None), (None,None), (None, None), ("label",LABEL)]) #,


        # Count the number of words and sentences.
        n_tokens_train = 0
        n_sentences_train = 0
        for ex in train_examples:
            n_tokens_train += len(ex.text) #+ 2
            n_sentences_train += 1
        n_tokens_valid = 0
        for ex in valid_examples:
            n_tokens_valid += len(ex.text)

        # Load the pre-trained embeddings that come with the torchtext library.
        use_pretrained = True
        if use_pretrained:
            print('We are using pre-trained word embeddings.')
            self.TEXT.build_vocab(train_examples, vectors="glove.840B.300d")
        else:
            print('We are training word embeddings from scratch.')
            self.TEXT.build_vocab(train_examples, max_size=5000)
        # self.LABEL.build_vocab(train_examples)

        # Create one of the models defined above.
        # self.model = RNNTagger(self.TEXT, self.LABEL, emb_dim=300, rnn_size=128, update_pretrained=False)
        # self.model = RNNCRFTagger(self.TEXT, self.LABEL, emb_dim=300, rnn_size=128, update_pretrained=False)

        self.model = TransformerEncoderModel(text_field=self.TEXT,
                                             label_field=self.LABEL,
                                             ntoken=300,
                                             d_model=300,
                                             d_hid=200,
                                             nlayers=2,
                                             nhead=2,
                                             dropout=0.2)
        self.model.to(self.device)

        # -----------------------------------------------------------
        #                       BucketIterator
        # -----------------------------------------------------------

        batch_size = 5 #1024
        n_batches = np.ceil(n_sentences_train / batch_size)

        mean_n_tokens = n_tokens_train / n_batches

        train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
                                                        (train_examples, valid_examples, test_examples),
                                                        device=self.device,
                                                        batch_size=batch_size,
                                                        sort_key=lambda x: len(x.text),
                                                        repeat=False,
                                                        sort=True)

        # train_iterator = data.BucketIterator(
        #     train_examples,
        #     device=self.device,
        #     batch_size=batch_size,
        #     sort_key=lambda x: len(x.text),
        #     repeat=False,
        #     train=True,
        #     sort=True)
        #
        # valid_iterator = data.BucketIterator(
        #     valid_examples,
        #     device=self.device,
        #     batch_size= 2, #64,
        #     sort_key=lambda x: len(x.text),
        #     repeat=False,
        #     train=False,
        #     sort=True)

        train_batches = train_iterator
        valid_batches = valid_iterator
        test_batches = valid_iterator


        # ---------------------------------------------------------------------
        #                       Optimizer
        # ---------------------------------------------------------------------
        criterion = nn.CrossEntropyLoss()
        lr = 5.0  # learning rate
        optimizer = torch.optim.SGD(self.model.parameters(), lr=lr)
        scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.95)
        # -------


        # optimizer = torch.optim.Adam(self.model.parameters(), lr=0.01, weight_decay=1e-5)

        # n_labels = len(self.LABEL.vocab)

        # ----------------------------------------------------------
        #                       Epoch Training
        # ----------------------------------------------------------

        history = defaultdict(list)

        n_epochs = 3 #25

        # For each epoch
        for i in range(1, n_epochs + 1):

            t0 = time.time()

            loss_sum = 0

            self.model.train()

            # for transformer
            scheduler.step()

            # For each batch
            for batch in train_batches:

                # Compute the output and loss.
                # loss = self.model(batch.text, batch.label) / mean_n_tokens

                out = self.model(batch.text)
                ntokens = 300
                loss = criterion(out.view(-1, ntokens), batch.label)

                optimizer.zero_grad()
                loss.backward()
                # I added this
                torch.nn.utils.clip_grad_norm_(self.model.parameters(), 0.5)
                # ---
                optimizer.step()
                loss_sum += loss.item()


            train_loss = loss_sum / n_batches
            history['train_loss'].append(train_loss)

            # Evaluate on the validation set.
            if i % 1 == 0:

                stats = defaultdict(Counter)

                # from transformers
                # lr = scheduler.get_last_lr()[0]
                # ms_per_batch = (time.time() - start_time) * 1000 / log_interval
                # cur_loss = total_loss / log_interval
                # ppl = math.exp(cur_loss)
                # print(f'| epoch {epoch:3d} | {batch:5d}/{num_batches:5d} batches | '
                #       f'lr {lr:02.2f} | ms/batch {ms_per_batch:5.2f} | '
                #       f'loss {cur_loss:5.2f} | ppl {ppl:8.2f}')
                # total_loss = 0

                self.model.eval()
                with torch.no_grad():
                    for batch in valid_batches:
                        # Predict the model's output on a batch.
                        predicted = self.model.predict(batch.text)
                        print("predicted := ", predicted)
                        # Update the evaluation statistics.
                        # evaluate_iob(predicted, batch.label, self.LABEL, stats)

                # # Compute the overall F-score for the validation set.
                # _, _, val_f1 = prf(stats['total'])
                #
                # history['val_f1'].append(val_f1)
                #
                # t1 = time.time()
                # print(f'Epoch {i}: train loss = {train_loss:.4f}, val f1: {val_f1:.4f}, time = {t1-t0:.4f}')

        # # After the final evaluation, we print more detailed evaluation statistics, including
        # # precision, recall, and F-scores for the different types of named entities.
        # print()
        # print('Final evaluation on the validation set:')
        # p, r, f1 = prf(stats['total'])
        # print(f'Overall: P = {p:.4f}, R = {r:.4f}, F1 = {f1:.4f}')
        # for label in stats:
        #     if label != 'total':
        #         p, r, f1 = prf(stats[label])
        #         print(f'{label:4s}: P = {p:.4f}, R = {r:.4f}, F1 = {f1:.4f}')
        #
        # plt.plot(history['train_loss'])
        # plt.plot(history['val_f1'])
        # plt.legend(['training loss', 'validation F-score'])

tagger = Tagger(lower=False)
tagger.train()

In [ ]:
def decode_params(spatial_params):
    l0 = spatial_params[0]
    alpha = spatial_params[1]
    alpha_rad = alpha * np.pi / 180
    l_i = spatial_params[2]
    beta_i = spatial_params[3]
    beta_i_rad = beta_i * np.pi / 180
    r = spatial_params[4]
    return l0, alpha, alpha_rad, l_i, beta_i, beta_i_rad, r


def point_in_space(spatial_params):
    l0, alpha, alpha_rad, l_i, beta_i, beta_i_rad, r = decode_params(spatial_params)
    # np.cos() and np.sin() take angles in radian as params
    center_pt = np.array([l0*np.cos(alpha_rad), l0 * np.sin(alpha_rad)])
    sense_pt = center_pt + np.array([l_i * np.cos(alpha_rad + beta_i_rad),
                                     l_i * np.sin(alpha_rad + beta_i_rad)])
    return sense_pt, center_pt


def inside_sphere(point, sphere_coo):

    pt = point_in_space(point)
    sphere_sense, sphere_center = point_in_space(sphere_coo)

    sphere_rad = sphere_coo[-1] # in angles

    contained = (pt[0] - sphere_sense[0])**2 + (pt[1] - sphere_sense[1])**2 <= sphere_rad**2

    if contained:
        return True
    else:
        return False

def distance(pred_pt, original_pt):
    """
    Calculates the distance between two sense points.
    :param pred_pt:
    :param original_pt:
    :return:
    """
    pred_sense, pred_center = point_in_space(pred_pt)
    orig_sense, orig_center = point_in_space(original_pt)

    return np.linalg.norm(pred_sense - orig_sense)



def sphere_dist(pred_pt, original_pt):
    """
    Calculates the distance between two 2D spheres.
    :param pred_pt:
    :param original_pt:
    :return:
    """
    pred_sense, pred_center = point_in_space(pred_pt)
    pred_radius = pred_pt[-1]
    orig_sense, orig_center = point_in_space(original_pt)
    orig_radius = original_pt[-1]

    return (pred_radius + orig_radius -
            np.linalg.norm(pred_sense - orig_sense))

def decode_prediction(spatial_params, df="SPATIAL_WORDNET.pickle") -> [str]:
    """
    Projects the predicted spatial parameters into the embedding space.
    Returns the synsets in the vacinity of the projected point.
    :param spatial_params:
    :return:
    """
    synsets = [] # sort from specific to most general

    sense_pt, center_pt = point_in_space(spatial_params)

    spatial_df = pd.read_pickle(df)
    # get the spheres, where the point/point+radius is contained/overlaping/near

    # 1. check if the predicted point is contained in some sense
    spatial_df["contained"] = spatial_df.apply(lambda row:
                                               inside_sphere(spatial_params,
                                                             row[['l0', 'alpha', 'l_i', 'beta_i', 'radius']]))

    # 2. For those synsets, which is the nearest synset point
    #use distance() to calculate distance between centers

    # 3. If None of the synsets apply to that word sense
    # use sphere_dist to find the nearest sphere (most general synset), and assign it to that synset
    # (this maybe good for rare senses)


    return synsets

def train_loss(tmp_pred, synset_params):
    # Loss is the distance between the two spheres/containment of the word within that sphere
    # radius acts as tolerance!
    return

In [ ]:
TEXT = data.Field(use_vocab=True,
                  lower=True)

LABEL = data.Field(is_target=True,
                   use_vocab=False,
                   unk_token=None,
                   sequential=False,
                   postprocessing=data.Pipeline(
                       lambda x: torch.tensor(list(map(float, removeBra(x).split(' '))),
                                              dtype=torch.double)),
                   dtype=torch.DoubleTensor)
                        #data.Pipeline(lambda x: torch.tensor(x, dtype=torch.double)))

# LABEL.numericalize()
fields = [("text",TEXT),("lemmatized_text",TEXT), (None, None), (None,None), (None, None), ("label",LABEL)]

In [ ]:
train, valid, test = datasets.SequenceTaggingDataset.splits(path='../data/test_transformer/',
                                   train = train_path,
                                   validation = validate_path,
                                   test = test_path,
                                   fields=[("text",TEXT),("lemmatized_text",TEXT), (None, None), (None,None), (None, None), ("label",LABEL)]) #,

In [ ]:
BATCH_SIZE = 5

use_pretrained = True
if use_pretrained:
    print('We are using pre-trained word embeddings.')
    TEXT.build_vocab(train, vectors="glove.840B.300d")
else:
    print('We are training word embeddings from scratch.')
    TEXT.build_vocab(train, max_size=5000)

In [ ]:
# LABEL.build_vocab(train.label)
LABEL.numericalize(train)

In [ ]:
train_iterator, valid_iterator = data.BucketIterator.splits(
                                                        (train, valid),
                                                        device=DEVICE,
                                                        batch_size=BATCH_SIZE,
                                                        sort_key=lambda x: len(x.text),
                                                        repeat=False,
                                                        sort=True)

In [ ]:
batch = next(iter(train_iterator))
print("Numericalize premises:\n", batch.text)
print("Entailment labels:\n", batch.label)